In [2]:
from bert_serving.client import BertClient
bc = BertClient(ip='192.168.0.105')

In [4]:
ans = bc.encode(['你好'])

In [20]:
ans2 = bc.encode(['"哎，想当年来佘山的时候，啥都没有，三品香算镇上最大看起来最像样的饭店了。菜品多，有点太多，感觉啥都有，杂都不足以形容。随便点些，居然口味什么的都好还可以，价钱自然是便宜当震惊。元宝虾和椒盐九肚鱼都不错吃。不过近来几次么，味道明显没以前好了。冷餐里面一个凉拌海带丝还可以，酸酸甜甜的。镇上也有了些别的大点的饭店，所以不是每次必来了。对了，这家的生意一如既往的超级好，不定位基本吃不到。不过佘山这边的人吃晚饭很早的，所以稍微晚点去就很空了。"'])

In [24]:
import numpy as np
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

basemodel.ipynb
bert_client.ipynb
comment-classification
comment-classification.zip
gru_cnn_model.ipynb
input
models
preprocess.ipynb
processed



In [30]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, Conv1D, MaxPool1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization

In [45]:
train_x = pd.read_csv('./input/sentiment_analysis_trainingset.csv')
train_y_processed = pd.read_csv('./processed/train_y_processed')
val_x = pd.read_csv('./input/sentiment_analysis_validationset.csv')
val_y_processed = pd.read_csv('./processed/val_y_processed')

In [47]:
train_list = train_x.content.tolist()
val_list = val_x.content.tolist()

In [55]:
train_bert_sentence_embedding1 = bc.encode(train_list[:10])

/Users/xincj/.virtualenvs/python3env/lib/python3.6/site-packages/bert_serving/client/__init__.py:285: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [64]:
data = pd.read_csv("file_name.csv", header=None)

In [65]:
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.902295,0.050484,-0.172733,0.163340,0.216406,-0.884055,-0.113720,0.193803,-0.024965,0.047187,...,-0.007610,0.245897,-0.025347,-0.055929,-0.028406,0.041145,0.058657,0.177345,0.279107,-0.214845
1,0.870421,0.025380,-0.282298,0.084426,0.207569,-0.944237,-0.350477,0.266563,-0.001834,-0.254577,...,-0.124792,0.022239,-0.065761,0.246078,-0.279375,-0.025940,0.330564,0.122222,0.287852,-0.305351
2,1.191080,0.005152,-0.711179,-0.109769,0.249980,-1.400326,-0.272405,0.510504,-0.163640,-0.549664,...,-0.312192,-0.321882,-0.142657,0.606598,-0.177795,0.178407,0.043636,0.180464,0.022202,-0.222446
3,0.843061,0.215399,-0.363675,0.097502,0.104311,-1.285999,-0.267110,0.339181,-0.332437,-0.306840,...,-0.291819,-0.133949,0.081637,0.067365,-0.226054,0.288608,0.105808,0.201949,0.300128,-0.199779
4,0.994475,0.233835,-0.323046,0.141066,-0.093911,-1.003276,-0.222923,0.478165,0.023107,-0.218585,...,-0.003418,-0.036019,-0.388153,0.104437,-0.101356,0.112896,0.107625,0.181771,0.284554,0.082852
5,1.085047,-0.049393,-0.447100,-0.063946,0.213721,-1.007541,-0.149813,0.515427,-0.276237,-0.345025,...,-0.040773,-0.094097,-0.360262,0.170443,-0.122729,0.188749,-0.014536,0.186082,0.372076,-0.184510
6,0.778790,0.072899,-0.060067,0.271493,0.097915,-0.995104,-0.063503,0.376292,-0.293974,-0.393685,...,0.108379,0.057622,-0.166568,0.269459,-0.135326,0.206665,0.169650,0.282393,0.359678,-0.222400
7,0.702339,0.137238,-0.330530,-0.123078,0.157724,-0.991845,-0.122050,0.106353,-0.138912,-0.366143,...,-0.209375,0.172143,-0.152252,0.232445,-0.103964,0.230836,0.130483,0.080095,0.105732,-0.224641
8,0.942434,0.028669,-0.158299,0.172286,0.100960,-0.724702,-0.166792,0.044498,0.031871,-0.142305,...,-0.028930,0.142949,0.047249,0.405469,-0.310105,-0.002172,0.031438,0.079474,0.242368,-0.103473
9,0.803228,-0.192137,0.031246,-0.078509,0.312756,-0.756581,-0.256271,0.288060,0.019070,-0.298895,...,0.226157,0.228943,-0.068472,0.222720,-0.396131,0.099331,0.103129,0.118500,0.195484,-0.088543


In [70]:
for i in range(0,1050):
    train_bert_sentence_embedding1 = batch = bc.encode(train_list[i*100: i*100 + 100])
    np.savetxt("./bert_embedding/{}.csv".format(i), train_bert_sentence_embedding, delimiter=",")

/Users/xincj/.virtualenvs/python3env/lib/python3.6/site-packages/bert_serving/client/__init__.py:285: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


KeyboardInterrupt: 

In [29]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [95]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(768,)))
# model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(80, activation="sigmoid"))
model.compile('adam', 'binary_crossentropy', metrics=['accuracy', f1])

In [96]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               393728    
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_9 (Dense)              (None, 80)                41040     
Total params: 697,424
Trainable params: 697,424
Non-trainable params: 0
_________________________________________________________________


In [76]:
train_x = []
for i in range(0, 250):
    data = pd.read_csv("./bert_embedding/{}.csv".format(i), header=None)
    train_x.append(data)

In [77]:
bert_embeddings_train = pd.concat(train_x)

In [84]:
label_train = train_y_processed.iloc[0:25000]

In [85]:
val_x = []
for i in range(250, 290):
    data = pd.read_csv("./bert_embedding/{}.csv".format(i), header=None)
    val_x.append(data)

In [86]:
bert_embeddings_val = pd.concat(val_x)

In [89]:
label_val = train_y_processed.iloc[25000:29000]

In [90]:
label_val.shape

(4000, 80)

In [97]:
batch_size = 32
epochs = 16

model.fit(bert_embeddings_train, label_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=[bert_embeddings_val, label_val])

Train on 25000 samples, validate on 4000 samples
Epoch 1/16
25000/25000 [==============================] - 9s 352us/step - loss: 0.3634 - acc: 0.8422 - f1: 0.6658 - val_loss: 0.3615 - val_acc: 0.8426 - val_f1: 0.6601
Epoch 2/16
25000/25000 [==============================] - 8s 339us/step - loss: 0.3607 - acc: 0.8434 - f1: 0.6698 - val_loss: 0.3621 - val_acc: 0.8420 - val_f1: 0.6534
Epoch 3/16
25000/25000 [==============================] - 9s 341us/step - loss: 0.3604 - acc: 0.8433 - f1: 0.6693 - val_loss: 0.3614 - val_acc: 0.8429 - val_f1: 0.6778
Epoch 4/16
25000/25000 [==============================] - 9s 340us/step - loss: 0.3600 - acc: 0.8434 - f1: 0.6706 - val_loss: 0.3612 - val_acc: 0.8427 - val_f1: 0.6692
Epoch 5/16
25000/25000 [==============================] - 9s 342us/step - loss: 0.3599 - acc: 0.8434 - f1: 0.6719 - val_loss: 0.3618 - val_acc: 0.8424 - val_f1: 0.6656
Epoch 6/16
25000/25000 [==============================] - 9s 341us/step - loss: 0.3598 - acc: 0.8434 - f1: 0.67